# Interaction with the contract

### Web3 instanciation and setup

**NOTE:** Before running this, ensure the blockchain is up by running `deploy.sh`.  

In [37]:
from dotenv import dotenv_values
from functools import wraps
from web3 import Web3
import json

url = "http://127.0.0.1:8545"

# Output from the deploy
contract_address = "0xcf94DD98F88B4975C3bb7074A213140D985FFe57"


env = dotenv_values(".env")
password = env["ACCOUNT_PASSWORD"]

# Also output from deploy
funder_addresses = [
    "0x548AEC5436F39e202205A2f829eeC592E12E7289",
    "0x87834f4A4c629745603e0DC338271246f1798c83",
]
DEFAULT_AMOUNT_FUND = 0.5 # ETH
UNLOCK_DURATION = 300 # Seconds

w3 = Web3(Web3.HTTPProvider(url))

In [38]:
def fund_new_wallet_decorator(func):
    """Wrapper function to automatically send funds from funder adresses to newly created accounts."""

    @wraps(func)
    def wrapper(*args, **kwargs):
        new_wallet_address, new_wallet_private_key = func(*args, **kwargs)

        # Unlock pre-funder wallets
        for address in funder_addresses:
            w3.geth.personal.unlock_account(
                address, password, UNLOCK_DURATION
            )  # Unlock for 5 minutes

        gas_price = w3.eth.gas_price
        gas_limit = w3.eth.estimate_gas(
            {
                "to": new_wallet_address,
                "value": w3.to_wei(DEFAULT_AMOUNT_FUND, "ether"),
                "from": funder_addresses[0],
            }
        )

        for address in funder_addresses:
            transaction = {
                "to": new_wallet_address,
                "value": w3.to_wei(DEFAULT_AMOUNT_FUND, "ether"),
                "gas": gas_limit,
                "gasPrice": gas_price,
                "from": address,
            }
            txn_hash = w3.eth.send_transaction(transaction)
            receipt = w3.eth.wait_for_transaction_receipt(txn_hash)

        return new_wallet_address, new_wallet_private_key

    return wrapper

### Registering an account in the Blockchain

In [39]:
@fund_new_wallet_decorator
def create_new_wallet(password: str):
    """Creates a new wallet in the blockchain"""
    new_wallet = w3.eth.account.create(password)
    new_wallet_address = new_wallet.address
    new_wallet_private_key = new_wallet._private_key.hex()

    return new_wallet_address, new_wallet_private_key


# Usage Example
new_wallet_address, new_wallet_private_key = create_new_wallet("password")

print(f"New Wallet Address: {new_wallet_address}")
print(f"New Wallet Private Key: {new_wallet_private_key}")
print(f"New Wallet Balance: {w3.eth.get_balance(new_wallet_address)}")

New Wallet Address: 0x40f74b42036D63f47b2b8693AC1285c39244b3E0
New Wallet Private Key: 0x78dcf8440cb9ac2513c2515babd4b122637f382f642a970d38abb220335d30f8
New Wallet Balance: 1000000000000000000


### Interacting with the contract

In [54]:
with open("./build/contracts_Voting_sol_Voting.abi", "r") as f:
    contract_abi = json.load(f)
contract = w3.eth.contract(address=contract_address, abi=contract_abi)

# Election and District info

election_name = "97ayrtf78oseryg78st"
description = "XANDÃO OWWWWWWWWWWWNT AWNT "

# Creating an election
trans = {
    "from": new_wallet_address,
    "gas": 2000000,
    "gasPrice": w3.to_wei("20", "gwei"),
    "nonce": w3.eth.get_transaction_count(new_wallet_address),
}
tx = contract.functions.createElection(election_name, description).build_transaction(
    {
        "from": new_wallet_address,
        "gas": 2000000,
        "gasPrice": w3.to_wei("20", "gwei"),
        "nonce": w3.eth.get_transaction_count(new_wallet_address),
    }
)

signed_tx = w3.eth.account.sign_transaction(tx, private_key=new_wallet_private_key)
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

In [55]:
contract.functions.getAdminElections(new_wallet_address).call()[:-1]

[(0,
  'Eleição Municipal de Morrinhos',
  'Eleição oficial para prefeitos da melhor cidade da região',
  '0x40f74b42036D63f47b2b8693AC1285c39244b3E0',
  '0x0000000000000000000000000000000000000000',
  0,
  9999999999,
  False,
  1722437610,
  1722437610),
 (1,
  'Eleição Municipal de Morrinhos',
  'Eleição oficial para prefeitos da melhor cidade da região',
  '0x40f74b42036D63f47b2b8693AC1285c39244b3E0',
  '0x0000000000000000000000000000000000000000',
  0,
  9999999999,
  False,
  1722437639,
  1722437639),
 (2,
  'Tteste',
  'Easdgsdftyhdfgyjftgyhuo',
  '0x40f74b42036D63f47b2b8693AC1285c39244b3E0',
  '0x0000000000000000000000000000000000000000',
  0,
  9999999999,
  False,
  1722437710,
  1722437710),
 (3,
  '97ayrtf78oseryg78st',
  'XANDÃO OWWWWWWWWWWWNT AWNT ',
  '0x40f74b42036D63f47b2b8693AC1285c39244b3E0',
  '0x0000000000000000000000000000000000000000',
  0,
  9999999999,
  False,
  1722437753,
  1722437753)]

In [49]:
contract.functions.getElection(0).call()

(0,
 'Eleição Municipal de Morrinhos',
 'Eleição oficial para prefeitos da melhor cidade da região',
 '0x40f74b42036D63f47b2b8693AC1285c39244b3E0',
 '0x0000000000000000000000000000000000000000',
 0,
 9999999999,
 False,
 1722437610,
 1722437610)

In [ ]:
# tx = contract.functions.addVoter(
#     0, "Leonardo", "12345", new_wallet_address
# ).build_transaction(
#     {
#         "from": new_wallet_address,
#         "gas": 2000000,
#         "gasPrice": w3.to_wei("20", "gwei"),
#         "nonce": w3.eth.get_transaction_count(new_wallet_address),
#     }
# )

# signed_tx = w3.eth.account.sign_transaction(tx, private_key=new_wallet_private_key)
# tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
# tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)